Getting the Shakespeaare Data [Later change to harry potter or something else]

In [19]:
!pip install wget

In [20]:
import wget
wget.download("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt")

'input (2).txt'

Reading the daata

In [21]:
with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()

Length of the dataset

In [22]:
print("length of the dataset in characters:",len(text))

length of the dataset in characters: 1115394


first 1000 characters

In [23]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



Set of all characters

In [24]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Character Level Tokenizer (Later change it to Tiktoken(gpt-2))

In [25]:
#mapping from characters to integers
c_to_i = {ch:i for i,ch in enumerate(chars)}
i_to_c = {i:ch for i,ch in enumerate(chars)}

In [26]:
#functions to use mappers. seperated becasue mappers will be  changed later.
encode = lambda s: [c_to_i[c] for c in s]
decode = lambda l: ''.join([i_to_c[i] for i in l])

print(encode("Hello World"))
print(decode(encode("Hello World")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


Changing the encoded tokens of the data into a data tensor.

In [27]:
! pip install torch

In [28]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

Seperating the dataset into training and validation

In [29]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

Explanation of context and target

In [30]:
block_size = 8 #to make it less expensive to feed the data to the model
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [31]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is: {target}")

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


Batch Dimension

In [32]:
torch.manual_seed(1337)
batch_size = 4 #independent sequences
block_size = 8 #maximum context length

In [33]:
# function to create the batch data
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

In [34]:
#sampling a new batch and printing it.
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [35]:
#printing the context and the target
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"When the context is {context.tolist()} the target is {target}")

When the context is [24] the target is 43
When the context is [24, 43] the target is 58
When the context is [24, 43, 58] the target is 5
When the context is [24, 43, 58, 5] the target is 57
When the context is [24, 43, 58, 5, 57] the target is 1
When the context is [24, 43, 58, 5, 57, 1] the target is 46
When the context is [24, 43, 58, 5, 57, 1, 46] the target is 43
When the context is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
When the context is [44] the target is 53
When the context is [44, 53] the target is 56
When the context is [44, 53, 56] the target is 1
When the context is [44, 53, 56, 1] the target is 58
When the context is [44, 53, 56, 1, 58] the target is 46
When the context is [44, 53, 56, 1, 58, 46] the target is 39
When the context is [44, 53, 56, 1, 58, 46, 39] the target is 58
When the context is [44, 53, 56, 1, 58, 46, 39, 58] the target is 1
When the context is [52] the target is 58
When the context is [52, 58] the target is 1
When the context is [52, 58, 1] th

Bigram Language Model

In [36]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx)
        B,T,C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
